# AML Movie Recommender

## Installation Information
This is written using python 3.11. To ensure that this works properly, please ensure that the correct version is used.
Install all necessary requirements using the requirements.txt file using the following command
pip install -r ./requirements.txt

## Usage
To receive correct predictions is necessary to use the exact name of the movie as it is used in the training data. The easiest way currently is to copy
the name of the movie from the movies.csv file.
To interact with the model there are two possible ways. It is possible to give the model movies as a list (compare recommender_notebook, cell six), and then run the method.
But there is also a terminal controller that allowes the user to add movies via a simple terminal menu.
Running the scripy.py will automatically prepare the data, train the model and start the terminal menu. This will take time


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from typing import Dict, Text
import tensorflow_recommenders as tfrs
import sys
from IPython.display import clear_output


In [ ]:
# This is the small dataset
ds_mov = 'Data/ml-latest-small/movies.csv'
ds_rat = 'Data/ml-latest-small/ratings.csv'
ds_tags = 'Data/ml-latest-small/tags.csv'

# This is the entire dataset, because of it's size this will take a long while to train the model. 
# ds_mov = 'Data/ml-latest/movies.csv'
# ds_rat = 'Data/ml-latest/ratings.csv'
# ds_tags = 'Data/ml-latest/tags.csv'


df_mov = pd.read_csv(ds_mov)
df_rat = pd.read_csv(ds_rat)
df_tags = pd.read_csv(ds_tags)
df_model = pd.merge(df_rat, df_mov, on='movieId', how='left')
df_model = pd.merge(df_model, df_tags, on=['movieId', 'userId'], how='left')
df_mov['movieId'] = df_mov['movieId'].astype(str)
df_model['userId'] = df_model['userId'].astype(str)
df_model['movieId'] = df_model['movieId'].astype(str)
df_model['tag'] = df_model['tag'].fillna('').astype(str)
df_model = df_model.drop(columns=['timestamp_x', 'timestamp_y'])

In [ ]:
mov_feat = {key: tf.constant(value) for key, value in df_mov.to_dict(orient='list').items()}
movies = tf.data.Dataset.from_tensor_slices(mov_feat)

rat_feat = {key: tf.constant(value) for key, value in df_model.to_dict(orient='list').items()}
ratings = tf.data.Dataset.from_tensor_slices(rat_feat)

movies = movies.map(lambda x: x['title'])
model_data = ratings.map(lambda x: {'userId': x['userId'], 'title': x['title'], 'rating': x['rating'], 'tag': x['tag']})
shuffled = model_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# for example in movies.take(1):
#     print(example)

In [ ]:
class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_user_ids.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding)
        ])
        self.tag_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_tags.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_tags) + 1, embedding)
        ])

    def call(self, inputs):
        if isinstance(inputs, dict):
            user_embedding = self.user_embedding(inputs['userId'])
            tag_embedding = self.tag_embedding(inputs['tag'])
            return tf.concat([user_embedding, tag_embedding], axis=1)
        else:
            return self.user_embedding(inputs)


class MovieModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.movie_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_movie_titles.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding*2)
        ])
        
        
    def call(self, inputs):
        return self.movie_embedding(inputs)


class RecommenderModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        self.user_model = UserModel()
        self.movie_model = MovieModel()
        self.task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        ))
        
    def compute_loss(self, features, training=False):

        user_embeddings = self.user_model({'userId': features['userId'], 'tag': features['tag']})  
        movie_embeddings = self.movie_model(features['title'])

        return self.task(user_embeddings, movie_embeddings)

embedding = 32

unique_user_ids = np.unique([x.numpy().decode('utf-8') for x in model_data.map(lambda x: x['userId'])])
unique_movie_titles = np.unique([x.numpy().decode('utf-8') for x in movies])

unique_tags = np.unique([
    x.numpy().decode('utf-8') for x in model_data.map(lambda x: x['tag']) if x.numpy().decode('utf-8') != ''
])



model = RecommenderModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()


model.fit(cached_train, epochs=10)

In [ ]:
model.evaluate(cached_test, return_dict=True)

In [ ]:

def recommend_movies(model, movie_list, movie_dataset, top_k):
    movie_embeddings = model.movie_model(tf.constant(movie_list))
    
    all_movie_titles = [x.numpy().decode('utf-8') for x in movie_dataset]
    all_movie_embeddings = model.movie_model(tf.constant(all_movie_titles))
    
    scores = tf.linalg.matmul(movie_embeddings, all_movie_embeddings, transpose_b=True)
    avg_scores = tf.reduce_mean(scores, axis=0) 
    
    top_indices = tf.argsort(avg_scores, direction='DESCENDING').numpy()
    recommended_movies = [all_movie_titles[i] for i in top_indices if all_movie_titles[i] not in movie_list][:top_k]
    
    return recommended_movies

list = [['Toy Story (1995)', 'Jumanji (1995)'], 
        ['Jurassic Park (1993)', 'Jurassic Park III (2001)'], 
        ['Batman Begins (2005)', 'The Shawshank Redemption (1994)'], 
        ['Batman Begins (2005)'], ['Lord of the Rings: The Fellowship of the Ring, The (2001)'], 
        ['Lord of the Rings, Batman'], 
        ['Zero Dark Thirty (2012)', 'American Sniper (2014)']]

['Lord of the Rings, Batman'] # This will result in bad predictions because the model has no idea what Lord of the Rings or Batman is, it has to be the exact title 

for liked_movies in list:
    print(recommend_movies(model, liked_movies, movies, top_k=5))

In [ ]:
def terminal_controller():
    list_of_movies = []
    while True:
        print ('Welcome to the movie recommender system!')
        print ('Please enter what you want to do:')
        print ('1. Add a movie you like')
        print ('2. Get movie recommendations')
        print ('3. List your movies')
        print ('4. Clear screen')
        print ('5. Exit')
        print ('')
        sys.stdout.flush() 
        user_input = input('Enter your choice: ')
        if user_input == '1':
            movie = input('Enter the name of the movie you like: ')
            list_of_movies.append(movie)
            print (f'Added {movie} to your list of liked movies.')
            print ('')
        elif user_input == '2':
            recommendations = recommend_movies(model, list_of_movies, movies, top_k=5)
            print ('We recommend the following movies:')
            for i, movie in enumerate(recommendations):
                print (f'{i+1}. {movie}')
            print ('')
        elif user_input == '3':
            print ('You have liked the following movies:')
            for i, movie in enumerate(list_of_movies):
                print (f'{i+1}. {movie}')
            print ('')
        elif user_input == '4':
            clear_output()
        elif user_input == '5':
            print ('Thank you for using the movie recommender system!')
            break
        else:
            print ('Invalid input. Please try again.')
            print ('')


In [ ]:
terminal_controller()

### Not working!

The code below is supposed to also take in the genres of the movies. The genres are loaded into the dataset as a ragged Tensor, but embedding the genres results in an error. 
A ragged Tensor is a 3D Tensor, so it has to be reduced for the embedding proccess. Embedding  requires the shape of a Tensor to be of rank of 2, but it is 3. After reducing the tensor with reduce_mean the shape of the Tensor is of rank 1. I don't know what might cause this issue, and how to properly handle it. 

In [ ]:
# This is the small dataset
ds_mov = 'Data/ml-latest-small/movies.csv'
ds_rat = 'Data/ml-latest-small/ratings.csv'
ds_tags = 'Data/ml-latest-small/tags.csv'

# This is the entire dataset, because of it's size this will take a long while to train the model
# ds_mov = 'Data/ml-latest/movies.csv'
# ds_rat = 'Data/ml-latest/ratings.csv'
# ds_tags = 'Data/ml-latest/tags.csv'


df_mov = pd.read_csv(ds_mov)
df_rat = pd.read_csv(ds_rat)
df_tags = pd.read_csv(ds_tags)
df_model = pd.merge(df_rat, df_mov, on='movieId', how='left')
df_model = pd.merge(df_model, df_tags, on=['movieId', 'userId'], how='left')
df_mov['movieId'] = df_mov['movieId'].astype(str)
df_model['userId'] = df_model['userId'].astype(str)
df_model['movieId'] = df_model['movieId'].astype(str)
df_model['tag'] = df_model['tag'].fillna('').astype(str)
df_model = df_model.drop(columns=['timestamp_x', 'timestamp_y'])
df_model['genres'] = df_model['genres'].apply(lambda x: x.split('|'))

In [ ]:
mov_feat = {key: tf.constant(value) for key, value in df_mov.drop(columns=['genres']).to_dict(orient='list').items()}
mov_feat['genres'] = tf.ragged.constant(df_mov['genres'].tolist())


movies = tf.data.Dataset.from_tensor_slices(mov_feat)

mod_feat = {key: tf.constant(value) for key, value in df_model.drop(columns=['genres']).to_dict(orient='list').items()}
mod_feat['genres'] = tf.ragged.constant(df_model['genres'].tolist())
ratings = tf.data.Dataset.from_tensor_slices(mod_feat)

movies = movies.map(lambda x: {'title': x['title'], 'genres': x['genres']}, )
ratings = ratings.map(lambda x: {'userId': x['userId'], 'title': x['title'], 'rating': x['rating'], 'tag': x['tag'], 'genres': x['genres']})



for example in ratings.take(1):
    print(example)

In [ ]:
class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_user_ids.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding)
        ])
        self.tag_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_tags.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_tags) + 1, embedding)
        ])
                
    def call(self, inputs):
        if isinstance(inputs, dict):
            user_embedding = self.user_embedding(inputs['userId'])
            tag_embedding = self.tag_embedding(inputs['tag'])
            return tf.concat([user_embedding, tag_embedding], axis=1)
        else:
            return self.user_embedding(inputs)


class MovieModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.movie_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_movie_titles.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding)
        ])
        self.genres_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_genres.astype(str), mask_token=None),
            tf.keras.layers.Embedding(len(unique_genres) + 1, embedding)
        ])
        
    def call(self, inputs):
        title_embedding = self.movie_embedding(inputs['title'])
        genres_embedding = self.genres_embedding(inputs['genres'])
        # genres_embedding = tf.reduce_mean(genres_embedding, axis=1)
        
        return tf.concat([title_embedding, genres_embedding], axis=1)

class RecommenderModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        self.user_model = UserModel()
        self.movie_model = MovieModel()
        self.task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        ))
        
    def compute_loss(self, features, training=False):

        user_embeddings = self.user_model({'userId': features['userId'], 'tag': features['tag']})  
        movie_embeddings = self.movie_model({'title': features['title'], 'genres': features['genres']})

        return self.task(user_embeddings, movie_embeddings)

embedding = 32

unique_user_ids = np.unique([x.numpy().decode('utf-8') for x in ratings.map(lambda x: x['userId'])])
unique_movie_titles = np.unique([x.numpy().decode('utf-8') for x in movies.map(lambda x: x['title'])])
unique_genres = np.unique([x for x in df_mov['genres'].explode()])
unique_tags = np.unique([
    x.numpy().decode('utf-8') for x in ratings.map(lambda x: x['tag']) if x.numpy().decode('utf-8') != ''
])

model = RecommenderModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


cached_train = ratings.shuffle(80_000).batch(8192).cache()
model.fit(cached_train, epochs=5)